In [1]:
import cv2
import mediapipe as mp
import time
import dlib
import numpy as np
from threading import Thread

def detect_faces(frame, face_detection):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_detection.process(frame_rgb)

    if results.detections:
        return True, results.detections
    else:
        return False, None

def detect_hands(frame, hand_detection):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hand_detection.process(frame_rgb)

    if results.multi_hand_landmarks:
        return True, results.multi_hand_landmarks
    else:
        return False, None

def main():
    cap = cv2.VideoCapture(0)  

    if not cap.isOpened():
        print("Error: Could not open camera.")
        return

    mp_face_detection = mp.solutions.face_detection
    face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.5)

    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)

    font_1 = cv2.FONT_HERSHEY_SIMPLEX
    detector = dlib.get_frontal_face_detector()

    class VideoStream:
        def __init__(self, stream):
            self.video = cv2.VideoCapture(stream)
            self.video.set(cv2.CAP_PROP_FPS, 60)

            if not self.video.isOpened():
                print("Can't access the webcam stream.")
                exit(0)
            
            self.grabbed, self.frame = self.video.read()
            self.stopped = True
            self.thread = Thread(target=self.update)
            self.thread.daemon = True
        
        def start(self):
            self.stopped = False
            self.thread.start()

        def update(self):
            while True:
                if self.stopped:
                    break
            
                self.grabbed, self.frame = self.video.read()
            self.video.release()

        def read(self):
            return self.frame

        def stop(self):
            self.stopped = True

    video_stream = VideoStream(stream=0)
    video_stream.start()

    cv2.namedWindow("Face Detection", cv2.WINDOW_NORMAL)

    no_detection_start_time = None
    detection_threshold = 5 

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Couldn't read frame.")
            break

        frame = cv2.flip(frame, 1) 

        detected_face, detections = detect_faces(frame, face_detection)
        detected_hand, hand_landmarks = detect_hands(frame, hands)

        if detected_face and not detected_hand:
            display_text = "Sitting"
            no_detection_start_time = None
        elif detected_hand:
            display_text = "Namastey"
            no_detection_start_time = None
        else:
            if no_detection_start_time is None:
                no_detection_start_time = time.time()
            elif time.time() - no_detection_start_time > detection_threshold:
                display_text = "Head Down"
            else:
                display_text = "Neutral"

        if display_text:
            cv2.putText(frame, display_text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        if video_stream.stopped:
            break
        else:
            frame_display = frame.copy()
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            rects = detector(gray, 1)

            for i, face_rect in enumerate(rects):
                left = face_rect.left()
                top = face_rect.top()
                width = face_rect.right() - left
                height = face_rect.bottom() - top

                cv2.rectangle(frame_display, (left, top), (left+width, top+height), (0, 255, 0), 2)
                cv2.putText(frame_display, f"Face {i+1}", (left - 10, top - 10), font_1, 0.7, (0, 255, 0), 2, cv2.LINE_AA)

                frame_crop = frame[top + 10:top+height-100, left + 30: left+width - 20]
                
                if frame_crop.size > 0:
                    img_blur = cv2.GaussianBlur(np.array(frame_crop), (5,5), sigmaX=1.7, sigmaY=1.7)
                    edges = cv2.Canny(image=img_blur, threshold1=100, threshold2=200)

                   
                    cv2.rectangle(frame_display, (left, top+height), (left+width, top+height+40), (0, 255, 0), cv2.FILLED)
                    cv2.putText(frame_display, "Face Detected", (left+10, top+height+20), font_1, 0.65, (0, 0, 255), 2, cv2.LINE_AA)

            cv2.imshow("Face Detection", frame_display)

        delay = 0.1
        time.sleep(delay)

        key = cv2.waitKey(1)
        if key == ord('q'):
            break

    video_stream.stop()
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


C:\Users\abc\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
